## Import libraries and dependencies


In [1]:
import optimiseprime.data_prep as dp
import optimiseprime.data_analysis as da

import pandas as pd
import yfinance as yf
import quantstats as qs
qs.extend_pandas()

import datetime as dt

## Request user for portfolio details, and fetch price data from yfinance Yahoo! Finance API

In [2]:
portfolio_choice = ""
while portfolio_choice not in [1, 2]:
    try:
        portfolio_choice = int(input(         
            "To analyse an existing portfolio, type 1\n"
            "To analyse a hypothetical portfolio - type 2\n"
        )
                              )
    except Exception:
        print('Error: Invalid Choice.\n')
        
# Get portfolio data from user depending on choice:

ticker_list = []

while len(ticker_list) == 0:
    if portfolio_choice == 1:
        existing_portfolio = dp.get_existing_portfolio()
        ticker_list = []
        for key, value in existing_portfolio.items():
            ticker_list.append(key)
        ticker_list = pd.DataFrame(columns=ticker_list).add_suffix('-USD').columns.tolist()
    elif portfolio_choice == 2:
        ticker_list = dp.get_hypothetical_portfolio()
        # Request investment amount from user
        investment_amount = dp.get_investment_amt()
    if len(ticker_list) == 0:
        print("You have not entered any valid tickers.")


# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = dp.get_ticker_data(ticker_list)
portfolio_df.dropna(inplace = True)

# Print portfolio data for visual confirmation
if portfolio_choice == 1: 
    print(f"--------------------------")                      
    print(f"Existing Portfolio:")
    total_value = int()
    for ticker, units in existing_portfolio.items():
        value = portfolio_df[f"{ticker}-USD"].iloc[-1, 3] * units
        print(f"Value of {units} {ticker}: ${value:.2f}")
        total_value += value
    print(f"\nTotal portfolio value: ${total_value:.2f}\n")        
elif portfolio_choice == 2:
    print(f"--------------------------")                      
    print(f"Hypothetical Portfolio:")
    print(f"{[ticker.replace('-USD', '') for ticker in ticker_list]}")                   
    print(f"Investment amount:")
    print(f"${investment_amount:.2f}\n")

print(
    f"To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.\n"
    f"Earliest date for which price data is available for all cryptocurrencies in your portfolio: {dt.datetime.date(portfolio_df.index[0])}"
)
print(f"--------------------------")
    

To analyse an existing portfolio, type 1
To analyse a hypothetical portfolio - type 2
 XRP


Error: Invalid Choice.



To analyse an existing portfolio, type 1
To analyse a hypothetical portfolio - type 2
 2


Please enter the tickers of your cryptocurrencies one by one
Type 'done' when finished.


Ticker:  XRP
Ticker:  XDC
Ticker:  VET
Ticker:  LINK
Ticker:  ADA
Ticker:  MATIC
Ticker:  done
How much do you wish to invest in total?
(Please input amount without currency symbol)
 30000


--------------------------
Hypothetical Portfolio:
['XRP', 'XDC', 'VET', 'LINK', 'ADA', 'MATIC']
Investment amount:
$30000.00

To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.
Earliest date for which price data is available for all cryptocurrencies in your portfolio: 2019-04-29
--------------------------


In [3]:
portfolio_df.head()


XRP-USD                                                         \
                open      high       low     close      volume daily_return   
Date                                                                          
2019-04-29  0.297422  0.299848  0.291652  0.294289   890911784    -0.010547   
2019-04-30  0.294320  0.311631  0.293542  0.309476  1229331267     0.051606   
2019-05-01  0.309490  0.311585  0.302282  0.304696   914654614    -0.015445   
2019-05-02  0.304735  0.305553  0.302567  0.303239   890538619    -0.004782   
2019-05-03  0.303232  0.313298  0.301475  0.308222  1221399046     0.016433   

             XDC-USD                                ...   ADA-USD            \
                open      high       low     close  ...       low     close   
Date                                                ...                       
2019-04-29  0.000538  0.000673  0.000468  0.000575  ...  0.064204  0.064631   
2019-04-30  0.000569  0.000652  0.000566  0.000643  ...  0.064661  0.070346   
2019-05-01  0.000643  0.000722  0.000494  0.000502  ...  0.067533  0.069014   
2019-05-02  0.000500  0.000722  0.000463  0.000503  ...  0.068278  0.068579   
2019-05-03  0.000553  0.000585  0.000492  0.000571  ...  0.067652  0.069858   

                                  MATIC-USD                                \
              volume daily_return      open      high       low     close   
Date                                                                        
2019-04-29  69287156    -0.057734  0.004418  0.004492  0.003471  0.003471   
2019-04-30  62990602     0.088425  0.003462  0.005078  0.003458  0.004478   
2019-05-01  58865338    -0.018935  0.004440  0.004673  0.004071  0.004374   
2019-05-02  55122571    -0.006303  0.004366  0.004700  0.004096  0.004198   
2019-05-03  77726744     0.018650  0.004192  0.004396  0.003952  0.004256   

                                     
                volume daily_return  
Date                                 
2019-04-29   8267044.0    -0.214884  
2019-04-30  25072706.0     0.290118  
2019-05-01  10567352.0    -0.023225  
2019-05-02   6714908.0    -0.040238  
2019-05-03   5702394.0     0.013816  

[5 rows x 36 columns]

## Data Analyses

### Calculate ratios:
* Sharpe ratio
* Sortino ratio
* Adjusted sortino ratio
* Gain to Pain ratio

In [4]:
# Calculate each of the following risk-reward ratio types
sharpe = da.calculate_sharpe_ratio(ticker_list, portfolio_df)
sortino =  da.calculate_sortino_ratio(ticker_list, portfolio_df)
adjusted_sortino = da.calculate_adjusted_sortino(ticker_list, portfolio_df)
gain_pain_ratio = da.calculate_gain_pain_ratio(ticker_list, portfolio_df)


# Store all ratios into a dict
ratios_df = pd.DataFrame(
    {
    'sharpe': sharpe,
    'sortino': sortino,
    'adjusted_sortino': adjusted_sortino,
    'gain_pain_ratio': gain_pain_ratio,
    }
)

# Calculate proportion scores for each risk-reward metric

ratio_prop_score = da.calculate_proportion_score(ratios_df)

ratio_prop_score